# Bold Play vs. Cautious Play

- At Las Vegas, a man with 20 dollars needs 40
- He decides to try to make his money by playing roulette
- He chooses one of the following strategies
    - **Bold Play**
        - Bet the 20 dollars on "evens" all at once and quit when he wins/loses
            - **Note**: P(winning) = 18/38
    - **Cautious Play**
        - Bet on "evens" one dollar at a time, and continue until he wins/loses
        
- *Which strategy is better?*

_____

- So we know the probability of winning 40 dollars with the **Bold Play** strategy is just 18/38

- The tricky part is calculating the probability of winning with the **Cautious Play** strategy
    - This is the same as the Gambler's Ruin problem
        - We can solve this using the Markov chain math we used in the previous problem

In [2]:
import numpy as np
import pandas as pd

In [3]:
p = 18/38
q = 1 - p

In [5]:
list_states = np.arange(0, 41)
n_states = len(list_states)

In [18]:
df_diag_1 = pd.DataFrame(q*np.identity(n_states),
                        index=list_states,
                        columns=list_states).shift(-1, axis=1).fillna(0)

df_diag_2 = pd.DataFrame(p*np.identity(n_states),
                        index=list_states,
                        columns=list_states).shift(1, axis=1).fillna(0)

df_transition_matrix = df_diag_1 + df_diag_2
df_transition_matrix.loc[0] = 0
df_transition_matrix.loc[0, 0] = 1
df_transition_matrix.loc[40] = 0
df_transition_matrix.loc[40, 40] = 1
df_transition_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.526316,0.000000,0.473684,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.526316,0.000000,0.473684,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.526316,0.000000,0.473684,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.526316,0.000000,0.473684,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Rearranging it into its canonical form

In [7]:
list_canonical = list(range(1, 40)) + [0, 40]
df_transition_matrix = df_transition_matrix.loc[list_canonical, list_canonical]

In [10]:
Q = df_transition_matrix.iloc[:-2, :-2].copy()
R = df_transition_matrix.iloc[:-2, -2:].copy()

In [11]:
N = np.linalg.inv(np.identity(len(Q)) - Q.values)

In [14]:
B = pd.DataFrame(np.matmul(N, R), index = range(1, 40), columns = [0, 40])

In [16]:
B.loc[20]

0     0.891602
40    0.108398
Name: 20, dtype: float64

- As we can see, his probability of finally getting to 40 dollars with the **Cautious Play** strategy is only 0.108398
    - **Therefore Bold Play is a way better strategy**

_____

- Let's try to tie out to our result with simulations

In [19]:
def gambling():
    total = 20
    while True:
        random = np.random.random()
        if random < 18/38:
            total += 1
        else:
            total -= 1
        if total == 0:
            return 0
        if total == 40:
            return 1

In [20]:
N_trials = 100000
list_results = []

for _ in range(N_trials):
    list_results.append(gambling())
np.mean(list_results)

0.10905

- Eyyyy